In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as ply
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [41]:
df=pd.read_csv('cardekho_imputated.csv')
df.head()

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15411 entries, 0 to 15410
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         15411 non-null  int64  
 1   car_name           15411 non-null  object 
 2   brand              15411 non-null  object 
 3   model              15411 non-null  object 
 4   vehicle_age        15411 non-null  int64  
 5   km_driven          15411 non-null  int64  
 6   seller_type        15411 non-null  object 
 7   fuel_type          15411 non-null  object 
 8   transmission_type  15411 non-null  object 
 9   mileage            15411 non-null  float64
 10  engine             15411 non-null  int64  
 11  max_power          15411 non-null  float64
 12  seats              15411 non-null  int64  
 13  selling_price      15411 non-null  int64  
dtypes: float64(2), int64(6), object(6)
memory usage: 1.6+ MB


### Handle Missing Value

In [43]:
df.isnull().sum()

Unnamed: 0           0
car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

### Feature Extraction

In [44]:
df.columns

Index(['Unnamed: 0', 'car_name', 'brand', 'model', 'vehicle_age', 'km_driven',
       'seller_type', 'fuel_type', 'transmission_type', 'mileage', 'engine',
       'max_power', 'seats', 'selling_price'],
      dtype='object')

In [45]:
df.drop(columns=['car_name','brand','Unnamed: 0'],axis=1,inplace=True)

In [46]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


### Train Test Split And Model Training

In [47]:
X=df.drop(columns=['selling_price'],axis=1)
Y=df['selling_price']

In [48]:
from sklearn.model_selection  import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=.3,random_state=42)

### Apply Standardization

In [49]:
numerical_features=[column for column in X_train.columns if X_train[column].dtype != 'object']

In [50]:
from sklearn.preprocessing import StandardScaler
for feature in numerical_features:
	scaler=StandardScaler()
	X_train[feature]=scaler.fit_transform(X_train[[feature]])
	X_test[feature]=scaler.transform(X_test[[feature]])

Label Encoding

In [51]:
categorical_feature=[column for column in X_train.columns if X_train[column].dtype == 'object']

In [52]:
from sklearn.preprocessing import LabelEncoder
for feature in categorical_feature:
	encoder=LabelEncoder()
	encoder.fit(X[feature])
	X_train[feature]=encoder.transform(X_train[feature])
	X_test[feature]=encoder.transform(X_test[feature])

### Model Training

In [53]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [54]:
# Evaluate the model
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [55]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
   
}

In [57]:
for model_name,model in models.items():
    model.fit(X_train, Y_train) 

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(Y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(Y_test, y_test_pred)

    
    print(model_name)
    
    print(f'Model performance for Training set')
    print(f"- Root Mean Squared Error: {model_train_rmse}")
    print(f"- Mean Absolute Error: {model_train_mae}")
    print(f"- R2 Score: {model_train_r2}")

    print('----------------------------------')
    
    print(f'Model performance for Test set')
    print(f"- Root Mean Squared Error: {model_test_rmse}")
    print(f"- Mean Absolute Error: {model_test_mae}")
    print(f"- R2 Score: {model_test_r2}")
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 560327.6024971509
- Mean Absolute Error: 268807.69236392365
- R2 Score: 0.6169165412402811
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 508110.85428486124
- Mean Absolute Error: 281212.3883644902
- R2 Score: 0.656708901551605


Lasso
Model performance for Training set
- Root Mean Squared Error: 560327.602515978
- Mean Absolute Error: 268807.4054926669
- R2 Score: 0.6169165412145377
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 508110.75019937154
- Mean Absolute Error: 281212.0238613782
- R2 Score: 0.6567090421965769


Ridge
Model performance for Training set
- Root Mean Squared Error: 560327.6151894087
- Mean Absolute Error: 268785.97560823575
- R2 Score: 0.6169165238854548
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 508105.7461979624
- Mean Absolute Error: 281191.2

In [58]:
knn_params = {"n_neighbors": [2, 3, 10, 20, 40, 50]}
rf_params = {"max_depth": [5, 8, 15, None, 10],
             "max_features": [5, 7, "auto", 8],
             "min_samples_split": [2, 8, 15, 20],
             "n_estimators": [100, 200, 500, 1000]}

In [59]:
randomcv_models=[
	("K-Neighbors Regressor", KNeighborsRegressor(),knn_params),
	("Random Forest Regressor", RandomForestRegressor(),rf_params)
]

In [63]:
from sklearn.model_selection import RandomizedSearchCV

models_best_params={}
for name,model,params in randomcv_models:
	random=RandomizedSearchCV(estimator=model,param_distributions=params,cv=5)

	random.fit(X_train,Y_train)
	models_best_params[name]=random.best_params_

for model_name,best_params in models_best_params.items():
	print(f"---------------- Best Params for {model_name} -------------------")
	print(best_params)

---------------- Best Params for K-Neighbors Regressor -------------------
{'n_neighbors': 2}
---------------- Best Params for Random Forest Regressor -------------------
{'n_estimators': 100, 'min_samples_split': 8, 'max_features': 5, 'max_depth': 15}


In [64]:
models = {
    "K-Neighbors Regressor": KNeighborsRegressor(n_neighbors=2),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=100,min_samples_split=8,max_features=5,max_depth=15)
}

In [65]:
for model_name,model in models.items():
    model.fit(X_train, Y_train) 

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(Y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(Y_test, y_test_pred)

    
    print(model_name)
    
    print(f'Model performance for Training set')
    print(f"- Root Mean Squared Error: {model_train_rmse}")
    print(f"- Mean Absolute Error: {model_train_mae}")
    print(f"- R2 Score: {model_train_r2}")

    print('----------------------------------')
    
    print(f'Model performance for Test set')
    print(f"- Root Mean Squared Error: {model_test_rmse}")
    print(f"- Mean Absolute Error: {model_test_mae}")
    print(f"- R2 Score: {model_test_r2}")
    
    print('='*35)
    print('\n')

K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 188179.94059827676
- Mean Absolute Error: 62570.594233799944
- R2 Score: 0.9567928102422664
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 306200.8950606584
- Mean Absolute Error: 116217.26319204153
- R2 Score: 0.8753309904551456


Random Forest Regressor
Model performance for Training set
- Root Mean Squared Error: 193775.18857275374
- Mean Absolute Error: 70656.79451302979
- R2 Score: 0.9541852097692545
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 244083.68020276225
- Mean Absolute Error: 104275.33490086366
- R2 Score: 0.9207821442085669


